In [1]:
import google.generativeai as genai
import re
import json
import traceback

/Users/bhavishyapandit/VSCProjects/google-ai-hackathon24/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def generate_response(prompt, temperature=0, safety_setting='BLOCK_NONE'):
    """
    Generates a resopnse by hitting to Gemini

    Parameters:
    - prompt (str): Description of the table.
    - temperature: The DataFrame containing the file data.

    Returns:
    - dict: Data dictionary containing the description of the table, each column, and its data type.
    """
    generation_config = {
      "temperature": temperature,
      "top_p": 1,
      "top_k": 1,
    }
    safety_settings = [
        {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": safety_setting
        },
        {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": safety_setting
        },
        {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": safety_setting
        },
        {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": safety_setting
        },
    ]
    genai.configure(api_key=gemini_token['key'])
    model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                                    generation_config=generation_config,
                                    safety_settings=safety_settings)
    convo = model.start_chat(history=[])
    convo.send_message(prompt)
    return re.sub(r"\*\*([^*]+)\*\*", r"\1", convo.last.text)

def auto_debugger(prompt, current_output, expected_output, temperature):
    user_prompt = f'''
        You are a helpful assisstant! Rewrite the below prompt so that the current output matches the expected output.
        Prompt: {prompt}
        Current Output: {current_output}
        Expected Output: {expected_output}

        Possible Solution(s):
        1. Role: Assign a role to a LLM to make sure it behaves a certain way
        2. Task: Clear explanation of the specific task to perform
        3. Instructions: Certain instructions to abide by (in bullet points) to ensure the response generated is concise and relevant to the prompt
        4. Output Format: An expected output format is shared (not the actual output)
        5. Examples: Example that shows flow - from input to the LLM generated output where all the instructions are followed.

        Adhere to below instructions at all costs!
        Instruction:
        1. Identify the cause of the error and rewrite the prompt - make it error free
        2. Don't include any irrelevant text, header or footer in your response
        3. Generate only refined prompt
        4. Follow these instructions by all means
        '''
    debugged_code = generate_response(user_prompt, temperature=temperature)
    return debugged_code

def generate_good_prompt(prompt):
    perfect_prompt = '''
    In context to a Large Language Model (LLM) a prompt is an input or query that a user or a program gives to an LLM, in order to get a specific response from the model.
    A good prompt comprises of the following things:

    1. Role: Assign a role to a LLM to make sure it behaves a certain way
    2. Task: Clear explanation of the specific task to perform
    3. Instructions: Certain instructions to abide by (in bullet points) to ensure the response generated is concise and relevant to the prompt
    4. Output Format: An expected output format is shared (not the actual output)
    5. Examples: Example that shows flow - from input to the LLM generated output where all the instructions are followed.

    Generate a perfect prompt:
    '''

    response = generate_response(perfect_prompt+prompt)
    return response

def generate_bad_prompt(prompt):
    perfect_prompt = '''
    In context to a Large Language Model (LLM) a prompt is an input or query that a user or a program gives to an LLM, in order to get a specific response from the model.
    A bad prompt misses atleast one of the following things except for "Task" and can have spelling and grammatical mistakes:

    1. Role: Assign a role to a LLM to make sure it behaves a certain way
    2. Task: A specific task to perform
    3. Instructions: Certain instructions to abide by (in bullet points) to ensure the response generated is concise and relevant to the prompt
    4. Output Format: An expected output format is shared (not the actual output)
    5. Examples: Example that shows flow - from input to the LLM generated output where all the instructions are followed.

    Generate a bad prompt:
    '''

    response = generate_response(perfect_prompt+prompt)
    return response
    

In [3]:
# fetching credentials
file = open('credentials.json', 'r')
gemini_token = json.load(file)

In [4]:
request = f'Write a prompt that summarises text from given text in not more than 200 words.'
resp = generate_bad_prompt(request)
print(resp)

Summarize the following text in not more than 200 words:

[Insert text here]


In [5]:
request = f'Write a prompt that summarises text from given text in not more than 200 words.'
resp = generate_good_prompt(request)
print(resp)

Role: Summarizer

Task: Summarize the following text in not more than 200 words.

Instructions:

* Identify the key points and supporting details in the text.
* Condense the information while maintaining the main ideas.
* Use clear and concise language.
* Avoid unnecessary details or repetitions.

Output Format:

A concise summary of the text, no longer than 200 words.

Example:

Input:

The Large Language Model (LLM) is a powerful tool that can be used for a variety of tasks, including text summarization. LLMs are trained on massive datasets of text, which allows them to learn the patterns and structures of language. This knowledge enables them to generate summaries that are both accurate and informative.

Output:

LLMs are trained on vast text datasets, enabling them to understand language patterns and structures. This allows them to generate accurate and informative summaries by identifying key points and supporting details, condensing information, and using clear language.


In [6]:
about_john = '''
John age: 35, a Canadian national, is a software engineer with a meticulous personality.  He thrives on crafting clean, efficient code, ensuring every line serves a purpose. 
John's 10 years of experience building web applications have honed his skills in a vast programming language arsenal.  He isn't just proficient; he enjoys the intellectual 
challenge of mastering new languages and frameworks. This passion for innovation is a driving force in his career. John seeks opportunities to push boundaries and develop 
creative solutions to complex technical problems.  The satisfaction of seeing his code come to life as a functional, user-friendly application fuels his dedication. 

However, John's life isn't all ones and zeros.  Beyond the digital realm, he finds enjoyment in the physical world through competitive sports. The camaraderie and strategic 
elements of team sports like basketball provide a welcome contrast to the solitary focus of coding. When not on the court, John might be found in his garage, tinkering with 
electronics projects.  Taking apart old devices and repurposing components to create something new allows him to combine his technical expertise with a hands-on approach. 
This playful exploration keeps his mind sharp and fosters a sense of accomplishment outside the professional sphere.  John's life exemplifies a healthy balance between the 
meticulous problem-solving of a software engineer and the thrill of hands-on creation and physical activity.
'''

prompt = f'''Extract details like name, age, profession, nationality from the text.
Text: {about_john}'''

expected_output = '''{'Name':name, 
                'Age':age,
                'Profession':profession,
                'Nationality': nationality
                }'''

current_output = generate_response(prompt)
print(current_output)

| Name | Age | Profession | Nationality |
|---|---|---|---|
| John | 35 | Software Engineer | Canadian |


In [16]:
debugged_prompt = auto_debugger(prompt, current_output, expected_output, temperature=0.2)
debugged_response = generate_response(debugged_prompt+f'Text:{prompt}')
print(debugged_response)

```
{
    'Name': 'John',
    'Age': 35,
    'Profession': 'Software Engineer',
    'Nationality': 'Canadian'
}
```
